# Training a model with a GravyFlow Dataset

In this notebook we will use our generated dataset to train a keras model. We start with the needed imports: 

In [1]:
# Built-in imports
from typing import List, Dict
from pathlib import Path

# Dependency imports: 
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Dense, Flatten, Dropout, ELU
from tensorflow.keras.models import Model

# Import the GravyFlow module.
import gravyflow as gf

2024-08-20 11:24:53.881612: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Set up the environment using gf.env() and return a tf.distribute.Strategy object.
env : tf.distribute.Strategy = gf.env(
	memory_to_allocate_tf=5000
)

INFO:root:TensorFlow version: 2.12.1, CUDA version: 11.8
2024-08-20 11:25:12.087704: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2024-08-20 11:25:12.088302: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5000 MB memory:  -> device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:85:00.0, compute capability: 7.0
INFO:root:[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## Creating a TensorFlow dataset through composition:

Rather than generating a generic Python iterator, we can also use GravyFlow to create a custom TensorFlow dataset. This will give us the ability to utalise all the functionality provided by the TensorFlow dataset class, including seamless integration with keras models, whilst maintaining the ability to generate datasets quickly enough for real time training, only caching downloaded data segments.

First we will create a keras model, inspired by a model from the literature, found at Gabbard _et at._ here: https://link.aps.org/doi/10.1103/PhysRevLett.120.141103:

In [3]:
def create_gabbard(
        input_shape_onsource : int, 
        input_shape_offsource : int
    ) -> tf.keras.Model:
    
    # Define the inputs based on the dictionary keys and expected shapes
    # Replace `input_shape_onsource` and `input_shape_offsource` with the actual shapes
    onsource_input = Input(shape=input_shape_onsource, name="ONSOURCE")
    offsource_input = Input(shape=input_shape_offsource, name="OFFSOURCE")

    # Pass the inputs to your custom Whiten layer
    # Assuming your Whiten layer can handle multiple inputs
    whiten_output = gf.Whiten()([onsource_input, offsource_input])

    x = gf.Reshape()(whiten_output)
    
    # Convolutional and Pooling layers
    x = Conv1D(8, 64, padding='same', name="Convolutional_1")(x)
    x = ELU(name="ELU_1")(x)
    x = MaxPooling1D(pool_size=4, strides=4, name="Pooling_1", padding="same")(x)
    
    x = Conv1D(8, 32, padding='same', name="Convolutional_2")(x)
    x = ELU(name="ELU_2")(x)
    x = Conv1D(16, 32, padding='same', name="Convolutional_3")(x)
    x = ELU(name="ELU_3")(x)
    x = MaxPooling1D(pool_size=4, strides=4, name="Pooling_3", padding="same")(x)
    
    # Flatten layer
    x = Flatten(name="Flatten")(x)
    
    # Dense layers with dropout
    x = Dense(64, name="Dense_1")(x)
    x = ELU(name="ELU_7")(x)
    x = Dropout(0.5, name="Dropout_1")(x)
    
    x = Dense(64, name="Dense_2")(x)
    x = ELU(name="ELU_8")(x)
    x = Dropout(0.5, name="Dropout_2")(x)
    
    outputs = Dense(1, activation='sigmoid', name="INJECTION_MASKS")(x)
    
    # Create model
    model = Model(inputs=[onsource_input, offsource_input], outputs=outputs, name="custom")
    
    return model

Because we are only using one injection, we expect our output label to be a single value for each example, therefore we must adjust the dimensionality of the injection masks output with tensorflow datasets mapping functionality, we define the function we want to map to the dataset here:

In [4]:
def adjust_features(features, labels):
    labels['INJECTION_MASKS'] = labels['INJECTION_MASKS'][0]
    return features, labels

TensorFlow and keras requires that the model and training dataset are created in the same scope, and is quite strict about these limitations. Thus we will here create our dataset and our model in the same scope. Nominally, it is anticipated that GravyFlow will mostly be used in Python scripts, rather than notebooks, where this will not be a problem if everything is kept in the same TensorFlow strategy:

In [5]:
with env:
    # This object will be used to obtain real interferometer data based on specified parameters.
    ifo_data_obtainer : gf.IFODataObtainer = gf.IFODataObtainer(
        observing_runs=gf.ObservingRun.O3, # Specify the observing run (e.g., O3).
        data_quality=gf.DataQuality.BEST,  # Choose the quality of the data (e.g., BEST).
        data_labels=[                      # Define the types of data to include.
            gf.DataLabel.NOISE, 
            gf.DataLabel.GLITCHES
        ],
        segment_order=gf.SegmentOrder.RANDOM, # Order of segment retrieval (e.g., RANDOM).
        force_acquisition=True,               # Force the acquisition of new data.
        cache_segments=False                  # Choose not to cache the segments.
    )

    # Initialize the noise generator wrapper:
    # This wrapper will use the ifo_data_obtainer to generate real noise based on the specified parameters.
    noise: gf.NoiseObtainer = gf.NoiseObtainer(
        ifo_data_obtainer=ifo_data_obtainer, # Use the previously set up IFODataObtainer object.
        noise_type=gf.NoiseType.REAL,        # Specify the type of noise as REAL.
        ifos=gf.IFO.L1                       # Specify the interferometer (e.g., LIGO Livingston L1).
    )

    scaling_method : gf.ScalingMethod = gf.ScalingMethod(
        value=gf.Distribution(
            min_=8.0,
            max_=15.0,
            type_=gf.DistributionType.UNIFORM
        ),
        type_=gf.ScalingTypes.SNR
    )

    # Define a uniform distribution for the mass of the first object in solar masses.
    mass_1_distribution_msun : gf.Distribution = gf.Distribution(
        min_=10.0, 
        max_=60.0, 
        type_=gf.DistributionType.UNIFORM
    )

    # Define a uniform distribution for the mass of the second object in solar masses.
    mass_2_distribution_msun : gf.Distribution = gf.Distribution(
        min_=10.0, 
        max_=60.0, 
        type_=gf.DistributionType.UNIFORM
    )

    # Define a uniform distribution for the inclination of the binary system in radians.
    inclination_distribution_radians : gf.Distribution = gf.Distribution(
        min_=0.0, 
        max_=np.pi, 
        type_=gf.DistributionType.UNIFORM
    )

    # Initialize a PhenomD waveform generator with the defined distributions.
    # This generator will produce waveforms with randomly varied masses and inclination angles.
    phenom_d_generator : gf.WaveformGenerator = gf.cuPhenomDGenerator(
        mass_1_msun=mass_1_distribution_msun,
        mass_2_msun=mass_2_distribution_msun,
        inclination_radians=inclination_distribution_radians,
        scaling_method=scaling_method,
        injection_chance=0.5 # Set so half produced examples will not contain this signal
    )
    
    training_dataset : tf.data.Dataset = gf.Dataset(       
        noise_obtainer=noise,
        waveform_generators=phenom_d_generator,
        input_variables=[
            gf.ReturnVariables.ONSOURCE, 
            gf.ReturnVariables.OFFSOURCE, 
        ],
        output_variables=[
            gf.ReturnVariables.INJECTION_MASKS
        ]
    ).map(adjust_features)

    validation_dataset : tf.data.Dataset = gf.Dataset(       
        noise_obtainer=noise,
        waveform_generators=phenom_d_generator,
        seed=1001, # Implement different seed to generate different waveforms,
        group="validate", # Ensure noise is pulled from those marked for validation.
        input_variables=[
            gf.ReturnVariables.ONSOURCE, 
            gf.ReturnVariables.OFFSOURCE, 
        ],
        output_variables=[
            gf.ReturnVariables.INJECTION_MASKS
        ]
    ).map(adjust_features)

    testing_dataset : tf.data.Dataset = gf.Dataset(       
        noise_obtainer=noise,
        waveform_generators=phenom_d_generator,
        seed=1002, # Implement different seed to generate different waveforms,
        group="test", # Ensure noise is pulled from those marked for validation.
        input_variables=[
            gf.ReturnVariables.ONSOURCE, 
            gf.ReturnVariables.OFFSOURCE, 
        ],
        output_variables=[
            gf.ReturnVariables.INJECTION_MASKS
        ]
    ).map(adjust_features)
    
    for input_example, _ in training_dataset.take(1):
        input_shape_onsource = input_example["ONSOURCE"].shape[1:]  # Exclude batch dimension    
        input_shape_offsource = input_example["OFFSOURCE"].shape[1:] 

    model = create_gabbard(input_shape_onsource, input_shape_offsource)

    # Now you can print the model summary
    model.summary()
    
    # Model compilation
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',  # Or any other loss function appropriate for your task
        metrics=['accuracy']
    )

2024-08-20 11:25:12.293381: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2024-08-20 11:25:12.425619: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7fead4056760 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-08-20 11:25:12.425688: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): Tesla V100-SXM2-16GB, Compute Capability 7.0
2024-08-20 11:25:12.431223: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-20 11:25:12.463553: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2024-08-20 11:25:12.601592

Model: "custom"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 ONSOURCE (InputLayer)          [(None, 1, 4096)]    0           []                               
                                                                                                  
 OFFSOURCE (InputLayer)         [(None, 1, 32768)]   0           []                               
                                                                                                  
 whiten (Whiten)                (None, 1, 2048)      0           ['ONSOURCE[0][0]',               
                                                                  'OFFSOURCE[0][0]']              
                                                                                                  
 reshape (Reshape)              (None, 2048, 1)      0           ['whiten[0][0]']            

If the previous cell failed it is most likely because you attempted to run it twice within the same kernel session. The kernal must be restarted in order to generate a fresh TensorFlow stratergy and recompile the model.

Finally, we can train the model with our generated dataset:

In [7]:
examples_per_epoch : int = int(1E5)
num_validation_examples : int = int(1E4)
num_testing_examples : int = int(1E4)
with env: 
    history = model.fit(
        training_dataset,
        epochs=10,  # Number of epochs to train for
        steps_per_epoch=examples_per_epoch // gf.Defaults.num_examples_per_batch,
        validation_data=validation_dataset,
        validation_steps=num_validation_examples // gf.Defaults.num_examples_per_batch #Ensure this is set as dataset is uncapped
    )

    model.evaluate(
        testing_dataset, 
        steps=num_testing_examples // gf.Defaults.num_examples_per_batch
    )

2024-08-20 11:26:02.386405: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2024-08-20 11:26:02.510630: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Epoch 1/10


2024-08-20 11:26:04.889467: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INTERNAL: No function library is provided.
	 [[{{function_node __inference_psd_2529}}{{node PartitionedCall}}]]
2024-08-20 11:26:04.898519: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INTERNAL: No function library is provided.
	 [[{{function_node __inference_truncate_transfer_2663}}{{node PartitionedCall}}]]


3123/3125 [============================>.] - ETA: 0s - loss: 0.3642 - accuracy: 0.8318

2024-08-20 11:27:03.860129: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2024-08-20 11:27:03.884842: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


...

3125/3125 [==============================] - 70s 22ms/step - loss: 0.1988 - accuracy: 0.9261 - val_loss: 0.1566 - val_accuracy: 0.9436
Epoch 7/10
3125/3125 [==============================] - ETA: 0s - loss: 0.1830 - accuracy: 0.9338

2024-08-20 11:34:13.674018: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
INFO:root:Noise generation failed: 
Traceback: Traceback (most recent call last):
  File "/home/michael.norman/data_ad_infinitum/chapter_05_dragonn/gravyflow/gravyflow/src/dataset/dataset.py", line 234, in __next__
    onsource, offsource, gps_times = next(self.noise)
StopIteration

2024-08-20 11:34:20.505733: W tensorflow/core/framework/op_kernel.cc:1818] UNKNOWN: Exception: 
Traceback (most recent call last):

  File "/home/michael.norman/data_ad_infinitum/chapter_05_dragonn/gravyflow/gravyflow/src/dataset/dataset.py", line 234, in __next__
    onsource, offsource, gps_times = next(self.noise)

StopIteration


During handling of the above exception, another except

UnknownError: Graph execution error:

2 root error(s) found.
  (0) UNKNOWN:  Exception: 
Traceback (most recent call last):

  File "/home/michael.norman/data_ad_infinitum/chapter_05_dragonn/gravyflow/gravyflow/src/dataset/dataset.py", line 234, in __next__
    onsource, offsource, gps_times = next(self.noise)

StopIteration


During handling of the above exception, another exception occurred:


Traceback (most recent call last):

  File "/home/michael.norman/.conda/envs/gravyflow/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 267, in __call__
    ret = func(*args)

  File "/home/michael.norman/.conda/envs/gravyflow/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/home/michael.norman/.conda/envs/gravyflow/lib/python3.10/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  

d))

  File "/home/michael.norman/data_ad_infinitum/chapter_05_dragonn/gravyflow/gravyflow/src/dataset/dataset.py", line 237, in __next__
    raise Exception()

Exception


2024-08-20 11:34:20.631355: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): UNKNOWN: Exception: 
Traceback (most recent call last):

  File "/home/michael.norman/data_ad_infinitum/chapter_05_dragonn/gravyflow/gravyflow/src/dataset/dataset.py", line 234, in __next__
    onsource, offsource, gps_times = next(self.noise)

StopIteration


During handling of the above exception, another exception occurred:


Traceback (most recent call last):

  File "/home/michael.norman/.conda/envs/gravyflow/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 267, in __call__
    ret = func(*args)

  File "/home/michael.norman/.conda/envs/gravyflow/lib/python3.10/s

d))

  File "/home/michael.norman/data_ad_infinitum/chapter_05_dragonn/gravyflow/gravyflow/src/dataset/dataset.py", line 237, in __next__
    raise Exception()

Exception


2024-08-20 11:34:20.640759: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): UNKNOWN: Exception: 
Traceback (most recent call last):

  File "/home/michael.norman/data_ad_infinitum/chapter_05_dragonn/gravyflow/gravyflow/src/dataset/dataset.py", line 234, in __next__
    onsource, offsource, gps_times = next(self.noise)

StopIteration


During handling of the above exception, another exception occurred:


Traceback (most recent call last):

  File "/home/michael.norman/.conda/envs/gravyflow/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 267, in __call__
    ret = func(*args)

  File "/home/michael.norman/.conda/envs/gravyflow/lib/python3.10/s